In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import openpyxl
import os
import time
import math
import webbrowser
import numpy as np
import re
import sys
from pandas import DataFrame


chrome_path = "C:\Temp\chromedriver.exe"
options = webdriver.ChromeOptions()
#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
# 창 숨기는 옵션 추가
# options.add_argument("headless")

driver = webdriver.Chrome(chrome_path,options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", 
{"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})
url = "https://www.oliveyoung.co.kr/store/main/main.do?oy=0"
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()

In [12]:

prod = input("어떤 상품을 검색하겠습니까? ")
number = input("몇 개의 상품을 검색하시겠습니까? ")
prod_text = prod
pro_number = int(number)
driver.find_element(By.CLASS_NAME,"placeholder_area").click()
prod = driver.find_element(By.ID,"query")
prod.send_keys(prod_text)
time.sleep(1)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "searchSubmit"))).click()

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

KeyboardInterrupt: Interrupted by user

In [ ]:
#선케어 제품들  url 가져오기
brand_name_list=[]
prod_url_list = []
items = soup.find_all("div",attrs={"class":"prd_info"})
for item in items:
    brand_name=item.find("span",attrs={"class":"tx_brand"}).get_text()
    brand_name_list.append(brand_name)
    url=item.find('a')['href']
    prod_url_list.append(url)


In [6]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
folder_path = 'C:/Users/won/teamproject/'+ prod_text

In [6]:
for i in range(0,pro_number):  
    driver.get(prod_url_list[i])
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "reviewInfo"))).click()
    time.sleep(2)


    html_type = driver.page_source
    soup_type = BeautifulSoup(html_type,'html.parser')
    time.sleep(2)
    #상품이름 가져오기
    pos_prod_name_list=[]
    pos_prod_name=soup_type.find('p',attrs={'class':'prd_name'}).get_text()
    pos_prod_name= re.sub('\xa0','',pos_prod_name)
    
    #상품 이미지 저장하기
    try :
        prod_url=soup_type.find('img',attrs={'id':'mainImg'})['src']
        print(prod_url)
    except :
        pass
    else :
        img_name = 'C:/Users/won/teamproject/prod_img/'+ prod_text + str(i)+'.jpg'
        urllib.request.urlretrieve(prod_url,img_name)  
        time.sleep(3)

    try:
        #페이지 넘기기면서 데이터 가져오기(리뷰,평점,아이디)
        pos_num=1
        pos_user_name_list=[]
        pos_rate_list=[]
        pos_review_list=[]


        #리뷰 검색 필터
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,'btnOption'))).click()
        #5점
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[2]'))).click()
        time.sleep(2)
        #4점

        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[3]'))).click()    
        time.sleep(2)
        #확인 버튼
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))).click()
        time.sleep(2)

        html_type = driver.page_source
        soup_type = BeautifulSoup(html_type,'html.parser')
        time.sleep(2)

        for num in range(1,31):
            #사용자이름

            try:
                user_names=soup_type.find_all('a',attrs={'class':'id'})
                for user_name in user_names:
                    pos_user_name_list.append(user_name.get_text())
            except:        
                pos_user_name_list.append("err")

            #평점
            try:
                non_rates=soup_type.find_all('div',attrs={'class':'score_area'})
                for rates in non_rates:
                    rate = rates.find('span',attrs={'class':'point'}).get_text()
                    rate = re.findall(r'\d', rate)
                    pos_rate_list.append(rate[1])

            except:
                pos_rate_list.append("rate_err")

            #리뷰
            try:    
                reviews=soup_type.find_all('div',attrs={'class':'txt_inner'})
                for review in reviews:
                    review = re.compile('[가-힣]+').findall(review.get_text())
                    pos_review_list.append(review)
            except:
                pos_review_list.append("review_err")
            #페이지 넘기기    
            try:
                if(num % 10 == 0):
                    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'next'))).click()
                    time.sleep(1)
                    html_type = driver.page_source
                    soup_type = BeautifulSoup(html_type,'html.parser')
                    time.sleep(2)
                else:
                    page_element=soup_type.find('div',attrs={'class':'pageing'})
                    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "{}".format(num+1)))).click()
                    time.sleep(1)
                    html_type = driver.page_source
                    soup_type = BeautifulSoup(html_type,'html.parser')
                    time.sleep(2)
            except:
                break


        createFolder(folder_path)
        ### 리뷰 가져오기
        try:
            pos_data={"상품이름" : pos_prod_name,
                     "유저 ID" : pos_user_name_list,
                      "유저평점" : pos_rate_list,
                      '리뷰':pos_review_list

                     }

            pos_df = pd.DataFrame(pos_data)
            pos_fileName = "pos"+ str(i) + prod_text + '.csv'
            os.chdir(folder_path)
            pos_df.to_csv(pos_fileName, index=False, encoding='cp949')
        except:
            print("에러" + pos_prod_name)
            pass


    except:
        continue

https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0017/A00000017165018ko.jpg?l=ko
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0016/A00000016547127ko.jpg?l=ko
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0016/A00000016717208ko.jpg?l=ko
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0018/A00000018319503ko.jpg?l=ko
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0016/A00000016629514ko.jpg?l=ko


In [18]:
pos_df

,상품이름,유저 ID,유저평점,review
0,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),깡나봉,5,지금 통째 비우고 있는 인생템입니다 제형가볍고 묽은 제형이예요 물처럼 흘러내리지는 ...
1,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),도레미빵,4,어제 사서 오늘드림으로 받아봤는데 오늘 오특 제법 킹받네요발림성이 산뜻가벼운 타입이...
2,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),몽티,5,식물나라 산소수 가벼운 수분 젤 로션 더블 증정 기획 첫 구매 구매일 판매가 원 세...
3,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),엔드,4,휴가때 사용하려고 구매했어여 흡수도 잘되고 백탁도 없어서 좋아요
4,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),봄빛햇살,5,여름 휴가때 사용하려고 바디용 선크림 주문해써욤 스틱이나 스프레이로 사려다가 꼼꼼히...
...,...,...,...,...
295,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),깡냉이라이더,5,기존에 쓰고있던 선크림이라 재구매했어요화장도 잘먹고 순해서 좋아요수부지인 저한테 좀...
296,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),두루딱딱이,5,빨리 살 것을요 식물나라 썬젤만 오래 사용하기도 했는데 집이 얼굴 바디용 있어서 걍...
297,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),모모냥,5,진짜 순하고 이런 선크림을 이제야 제가 안게 너무 후회될 정도에요 이래서 위제품이구...
298,식물나라 산소수가벼운수분선젤로션 더블+증정기획(60+60+15ml)(2206),주룩,5,제 피부타입은 수분부족지성입니다 최근까지 써오던 선크림이 단종이 되어서 선크림 유목...


#리뷰 긍부정 키워드 나누기


In [2]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np
import re
from konlpy.tag import Mecab
import pandas as pd
import matplotlib as mpl
from gensim.models import Word2Vec
import numpy as np
from konlpy.tag import Okt
from tqdm import tqdm
import os
okt = Okt()

In [3]:
#긍정 데이터프레임 만들기(상품이름,리뷰 키워드 추출 후 저장)
#데이터 베이스에서 검색할 상품 검색
pos_prod_name_list=[]
prod=input("검색하실 상품")
prod_text=prod
folder_path = 'C:/Users/won/teamproject/'+ prod_text
os.chdir(folder_path)

검색하실 상품마스크팩


In [4]:
#키워드 추출 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keywords(text, n_keywords):
    # TF-IDF 벡터화 객체 생성
    tfidf_vectorizer = TfidfVectorizer()

    # 텍스트 데이터를 TF-IDF 행렬로 변환
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])

    # 단어 목록 가져오기
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    # Check if vocabulary is empty
    if len(feature_names) == 0:
        return []

    # TF-IDF 값이 큰 순서대로 정렬하여 상위 키워드 추출
    top_keywords = []
    if n_keywords > len(feature_names):
        n_keywords = len(feature_names)
    # Convert sparse matrix to dense array
    tfidf_array = tfidf_matrix.toarray()[0]

    # Get indices of top TF-IDF values
    top_indices = tfidf_array.argsort()[::-1][:n_keywords]

    # Get corresponding keywords
    top_keywords = [feature_names[idx] for idx in top_indices]

    return top_keywords

In [7]:
#긍정 데이터프레임 만들기(상품이름,리뷰 키워드 추출 후 저장)
pos_prod_name_list=[]
whole_pos_keywords_list=[]
keywords_list=[]
for a in range(0,5):
    pos_fileName="pos"+prod_text+str(a)+"keywords"
    pos_df = pd.read_csv("pos" + str(a) + prod_text + ".csv",encoding="cp949")                 
    pos_prod_name_list.append(pos_df.loc[1]['상품이름'])
    text=''
    review=[]

    for each_review in pos_df['리뷰']:
        each_review = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", each_review)
        review.append(each_review)

    pos_df = pos_df.drop(['리뷰'], axis=1)

    pos_df['review'] = review

    #명사 추출 과정-> 불필요한 단어를 제거하기 위한 리뷰 전체 키워드 저장
    
    for i in range(0,len(pos_df['review'])):

        whole_pos_keyword = okt.nouns(pos_df['review'][i])
        whole_pos_keywords_list.extend(whole_pos_keyword)

    for i in range(0,len(pos_df['review'])):

#         예시 텍스트
        text = pos_df['review'][i]



#         키워드 추출
        if(text != ''):
            keywords = extract_keywords(text, 10)

#         추출된 키워드 출력
        keywords_list.extend(keywords)
    
    #명사 추출 과정
    pos_noun_keywords_list=[]   
    for keyword in keywords_list:


        pos_keyword = okt.nouns(keyword)
        pos_noun_keywords_list.extend(pos_keyword)


    df_word_count = pd.DataFrame()
    word_list = list(set(pos_noun_keywords_list))

    word_cnt_list = []
    word_cloud_dict={}
    word_list_1=[]
    #word_list에 있는 키워드를 카운팅
    for word in tqdm(word_list):

        if len(word) != 1:
            word_list_1.append(word)
            word_cnt = pos_noun_keywords_list.count(word)
            word_cnt_list.append(word_cnt)
            word_cloud_dict[word] = word_cnt

        else:
            pass

    df_word_count['word'] = word_list_1
    df_word_count['word_cnt'] =  word_cnt_list
    df_word_count = df_word_count.sort_values('word_cnt',ascending=False)
    pos_kewords_filePath = folder_path+'/positive_keywords_tf/'
    createFolder(pos_kewords_filePath)
    df_word_count.to_csv(pos_kewords_filePath+pos_fileName+'.csv',index=False, encoding='cp949')
            
    



100%|████████████████████████████████████████████████████████████████████████████| 1248/1248 [00:00<00:00, 9868.06it/s]


In [8]:
df_word_count

,word,word_cnt
852,사용,357
795,마스크,334
0,피부,291
351,진정,150
707,구매,131
...,...,...
610,세번,1
609,단시간,1
318,에션스,1
319,몇몇,1


In [ ]:
pos_prod_name_list

In [9]:
#전체 리뷰 분석해서 제거해야 할 키워드 추출(키워드 추출 시 중복된 값이 없다)
pos_whole_word_list = list(set(whole_pos_keywords_list))

pos_whole_word_list_1=[]
pos_whole_word_cnt_list=[]
pos_whole_word_cloud_dict={}
print(brand_name_list[a])
    
for word in tqdm(pos_whole_word_list):
    
    if len(word) != 1:
        pos_whole_word_list_1.append(word)
        pos_whole_word_cnt = whole_pos_keywords_list.count(word)
        pos_whole_word_cnt_list.append(pos_whole_word_cnt)
        pos_whole_word_cloud_dict[word] = pos_whole_word_cnt
        

    else:
        pass

NameError: name 'brand_name_list' is not defined

In [ ]:
#제거해야할 긍정 키워드
pos_df_whole_word_count=pd.DataFrame()
pos_df_whole_word_count['pos_whole_word'] = pos_whole_word_list_1

pos_df_whole_word_count['pos_whole_word_cnt'] =  pos_whole_word_cnt_list

pos_df_whole_word_count = pos_df_whole_word_count.sort_values('pos_whole_word_cnt',ascending=False)


pos_df_whole_word_count[:30]


In [ ]:
#불용어 사전을 가져와서  제거 리스트에 추가
stop_words=[]
file = open("C:\\Users\\won\\teamproject\\stopwords.txt")
line=file.readline()
stop_words = line.split(',')

In [ ]:
pos_drop_keywordlist = ['마스크','피부','사용','제품','현상','시림','구매','느낌','추천','진짜','아주','선크림','한번','하나',\
                        '발라','얼굴','썬크림','약간','생각','효과','정도','살짝','세안','폼클렌징','클렌징폼','조금','정말']
pos_drop_keywordlist.extend(stop_words)
pos_drop_keywordlist.extend(brand_name_list)


In [ ]:
#긍정 불필요 키워드 제거
os.chdir(pos_kewords_filePath)
pos_ref_filePath = pos_kewords_filePath +"pos_ref/"
createFolder(pos_ref_filePath)
print(pos_ref_filePath)

        
        
for a in range(0,5):

    fileName="pos"+prod_text+str(a)+"keywords"
    pos_refined_df=pd.read_csv(fileName+'.csv',encoding="cp949")
    pos_refined_df.replace('정력','세정력',inplace=True)
    pos_refined_df.replace('발림','발림성',inplace=True)
    pos_refined_df.replace('트리','티트리',inplace=True)
    pos_drop_keywordlist.extend(okt.nouns(brand_name_list[a]))
    pos_drop_keywordlist .__contains__('마녀공장')
    for ref_word in pos_drop_keywordlist:

        pos_idx = pos_refined_df[pos_refined_df['word'] == ref_word].index

        pos_refined_df.drop(pos_idx,inplace=True)


    pos_refined_df['prod_name'] = pos_prod_name_list[a]
    pos_refined_df[:5].to_csv(pos_ref_filePath + fileName+'_ref.csv',index=False, encoding='cp949')
#     except:
#         continue
         

In [ ]:
pos_refined_df